## init

Since I am responsible for virtually all data updates to the database for `<plan name>`, there can be ~60+ logs each week to process. Some can be closed after the database has been updated, while others (deaths) need to be re-assigned for further processing. 

The update process has been more or less sufficiently streamlined allowing for bulk updates done in batches. Manually closing/re-assigning ~60+ logs at once was a hassle though, so I hacked together this script. It's not the fastest (delays intentionally added as the site can be glitchy sometimes) but it works error-free and allows me to do other things while it runs in the background.  

In [ ]:
import os
import time

import pandas as pd
import yaml
from loguru import logger
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

logger.add("loguru.log")

### fxs

#### init fxs

In [ ]:
def return_yaml_data(
    key: str = None, file_path: str = os.getcwd() + "\\" + "config.yaml"
):
    with open(file_path, "r") as f:
        data = yaml.safe_load(f.read())

    if key is None:
        return data

    try:
        result = data[key]
    except KeyError:
        return data

    return result


def search_for_xpath_elem(xpath_str: str, time_limit: int = 30) -> list:
    """While len of search list is 0, keep searching. If exceeds time limit (seconds, approximate),  returns empty list."""

    ctr = 0
    t_search = driver.find_elements(By.XPATH, xpath_str)

    while (len(t_search) == 0) and (ctr < time_limit):
        time.sleep(1)
        ctr += 1
        t_search = driver.find_elements(By.XPATH, xpath_str)

    if len(t_search) > 0:
        return t_search
    else:
        return []


def return_data() -> pd.DataFrame:
    latest = sorted(os.listdir("./data"))[-1]
    print(latest)

    df = pd.read_excel(f"./data/{latest}")
    df = df[["ticket_num", "death_update"]].copy()
    df.columns = ["ticket_num", "log_action"]

    return df

#### browser automation fxs

In [ ]:
def login():
    driver.get(HARMONY_URL)

    t_search = search_for_xpath_elem(return_yaml_data("username_xpath"))
    t_search[0].send_keys(USERNAME)

    t_search = search_for_xpath_elem(return_yaml_data("password_xpath"))
    t_search[0].send_keys(PASSWORD)
    time.sleep(1)
    t_search[0].send_keys(Keys.RETURN)

In [ ]:
def search_ticket_num() -> None:
    # click search by ticket number
    time.sleep(1)
    t_search = search_for_xpath_elem(return_yaml_data("log_search_xpath"))
    time.sleep(0.5)
    t_search[0].click()

    # refresh
    t_search = search_for_xpath_elem(return_yaml_data("refresh_btn_xpath"))
    time.sleep(1)
    t_search[0].click()

    # input ticket number
    t_search = search_for_xpath_elem(return_yaml_data("tkt_num_input_xpath"))
    time.sleep(1)
    t_search[0].send_keys(str(ticket_num))

    # click search
    t_search = search_for_xpath_elem(return_yaml_data("tkt_num_button_xpath"))
    time.sleep(1)
    t_search[0].click()

In [ ]:
def view_log() -> None:
    # there should only be one result
    t_search = search_for_xpath_elem(return_yaml_data("result_actions_xpath"))
    time.sleep(1)
    t_search[0].click()

    # view log
    t_search = search_for_xpath_elem(return_yaml_data("view_log_btn_xpath"))
    time.sleep(1)
    t_search[0].click()

In [ ]:
def click_edit_log() -> None:
    t_search = search_for_xpath_elem(return_yaml_data("log_edit_btn_xpath"))
    time.sleep(1)
    t_search[0].click()

In [ ]:
def edit_and_submit(comment="done"):
    # xpath for resolution input differs by log type
    xpath_list = return_yaml_data("resolution_input_xpath_list")
    t_search = []
    while len(t_search) == 0:
        for xpath_str in xpath_list:
            t_search = driver.find_elements(By.XPATH, xpath_str)
            time.sleep(0.5)
            if len(t_search) > 0:
                t_search[0].send_keys(comment)
                break  # exit out of loop once element found

    # xpath for submit button differs by log type
    xpath_list = return_yaml_data("submit_btn_xpath_list")
    t_search = []
    while len(t_search) == 0:
        for xpath_str in xpath_list:
            t_search = driver.find_elements(By.XPATH, xpath_str)
            if len(t_search) > 0:
                break  # exit out of loop once element found
    t_search[0].click()

In [ ]:
def reassign_log() -> None:
    # there should only be one result
    t_search = search_for_xpath_elem(return_yaml_data("result_actions_xpath"))
    time.sleep(0.5)
    t_search[0].click()

    # click re-assign
    t_search = search_for_xpath_elem(return_yaml_data("reassign_btn_xpath"))
    time.sleep(0.5)
    t_search[0].click()

    # enter name
    t_search = search_for_xpath_elem(return_yaml_data("search_user_xpath"))
    time.sleep(0.5)
    t_search[0].send_keys(ASSIGN_TO)
    time.sleep(0.3)
    t_search[0].send_keys(Keys.RETURN)

    # click assign button
    t_search = search_for_xpath_elem(return_yaml_data("assign_btn_xpath"))
    time.sleep(0.3)
    t_search[0].click()

    # enter reason
    t_search = search_for_xpath_elem(return_yaml_data("reason_input_xpath"))
    time.sleep(0.5)
    t_search[0].send_keys(note)

    # submit reason
    time.sleep(1.5)
    submit_reason_xpath = return_yaml_data("submit_reason_xpath")
    t_search = driver.find_elements(By.XPATH, submit_reason_xpath)
    t_search[0].click()

### login

In [ ]:
# private
USERNAME = return_yaml_data("user")
PASSWORD = return_yaml_data("pass")
HARMONY_URL = return_yaml_data("url")
ASSIGN_TO = return_yaml_data("colleague")

serv = Service(return_yaml_data("chromedriver"))
driver = webdriver.Chrome(service=serv)
login()

## run

### close logs

In [ ]:
df = return_data()
mask = df.ticket_num.isna()
if df[mask].shape[0] != 0:
    logger.warning("Missing ticket numbers in data")

# filter for only logs to close
mask = df["log_action"].str.strip().str.lower() == "n"
logs_to_close = df[mask].copy()
logs_to_close = logs_to_close.reset_index(drop=True).copy()
logger.info(f"Logs to close: {len(logs_to_close)}")

completed_list = []

In [ ]:
for i in range(len(logs_to_close)):
    ticket_num = logs_to_close.loc[i, "ticket_num"]

    logger.info(i)
    logger.info(ticket_num)
    logger.info(f"{i+1}/{len(logs_to_close)}")

    if ticket_num in completed_list:
        logger.info(f"already done {ticket_num}")
    else:
        search_ticket_num()
        view_log()
        click_edit_log()
        edit_and_submit()
        time.sleep(0.5)
        completed_list.append(ticket_num)
        logger.info(f"closed {ticket_num}")

### reassign logs

In [ ]:
df = return_data()
mask = df.ticket_num.isna()
if df[mask].shape[0] != 0:
    logger.warning("Missing ticket numbers in data")

# filter for only logs to re-assign
mask = df["log_action"].str.strip().str.lower() != "n"
logs_to_close = df[mask].copy()  # ignore this variable name
logs_to_close = logs_to_close.reset_index(drop=True).copy()
logger.info(f"Logs to re-assign: {len(logs_to_close)}")

completed = []

In [ ]:
for i in range(len(logs_to_close)):
    ticket_num = logs_to_close.loc[i]["ticket_num"]
    note = logs_to_close.loc[i]["log_action"]

    logger.info(i)
    logger.info(ticket_num)
    logger.info(f"{i+1}/{len(logs_to_close)}")

    if ticket_num in completed:
        logger.info(f"already done {ticket_num}")
    else:
        search_ticket_num()
        reassign_log()
        time.sleep(2.7)
        completed.append(ticket_num)
        logger.info(f"re-assigned {ticket_num} w/ note: {note}")